Author: <font color="blue">Josep Monclús Carrasco</font>

E-mail: <font color="blue">josep.monclus01@estudiant.upf.edu</font>

Date: <font color="blue">18/10/2021</font>

In [1]:
#We install apyori if we don't have it
#!pip install apyori

In [2]:
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd  
from apyori import apriori

# 1. Playing with apyori

In [3]:
#We crate the transactions
transactions = [
    ["chips","cola"],["water","almonds"],
    ["chips","water","beef"],["almonds","beef","chips"],
    ["chips","almonds"],["water","almonds","beef"],
    ["cola","chips"],["water","almonds"],
    ["chips"],["beef","chips"],
    ["chips","cola"],["cola","beef","chips"],
    ["water","almonds"],["almonds","chips"],
    ["water","cola"],["cola","beef"],
    ["almonds","water"],["beef","chips","cola"],
    ["cola"],["beef","cola"],["cola","chips"],["cola","chips","beef"]
]
results = list(apriori(transactions, min_support=0.2, min_confidence=0.7, min_lift=1))
print(results)

[RelationRecord(items=frozenset({'water', 'almonds'}), support=0.22727272727272727, ordered_statistics=[OrderedStatistic(items_base=frozenset({'water'}), items_add=frozenset({'almonds'}), confidence=0.7142857142857143, lift=1.9642857142857142)])]


In [4]:
# Leave this code as-is

def print_apyori_output (association_results):
    for relation_record in association_results:
        itemset = list(relation_record.items)
        
        # Consider only itemsets of two elements
        if len(itemset) > 1: 
        
            print("Rules involving itemset %s" % itemset)
            support = relation_record.support

            for rules in relation_record.ordered_statistics:
                antecedent = list(rules.items_base)
                consequent = list(rules.items_add)
                confidence = rules.confidence
                lift = rules.lift

                print("%s => %s (support=%.2f, confidence=%.2f, lift=%.2f)" %
                      (antecedent, consequent, support, confidence, lift))
            print()

In [5]:
print_apyori_output(results)

Rules involving itemset ['water', 'almonds']
['water'] => ['almonds'] (support=0.23, confidence=0.71, lift=1.96)



### Confidence

$ conf(water \Rightarrow almonds) = \frac {sup(water,almonds)}{sup(water)} $

$ sup(water,almonds) = \frac{5}{22} =  0.227 $ 

**[water,almonds] apears in 5 transactions (you only have to count) of the 22 total transactions**

$ sup(water) = \frac{7}{22} =  0.318 $ 

**[water] apears in 7 transactions of the 22 total**

$ conf(water \Rightarrow almonds) = \frac {\frac{5}{22}}{\frac{7}{22}} = $ **0.7142**


### Lift

$ lift(water \Rightarrow almonds) = \frac {sup(water,almonds)}{sup(water)\times sup(almond)} $

$ sup(water,almonds) = \frac{5}{22} =  0.227 $

$ sup(water) = \frac{7}{22} =  0.318  $

$ sup(almonds) = \frac{8}{22} =  0.363 $

$ lift (water \Rightarrow almonds) = \frac {\frac{5}{22}}{\frac{7}{22}\times\frac{8}{22}} = $ **1.9642** 

# 2. Load and prepare the services purchased dataset

In [6]:
#I change the file path from services_purchased.csv to data/services_purchased.csv
INPUT_FILENAME = "data/services_purchased.csv"

In [7]:
dataset = pd.read_csv(INPUT_FILENAME, sep=",")
dataset.head()

,ID_customer,WEBHOSTING,OFFICESUITE,SECURITY,CLOUD_IAAS,CLOUD_PAAS,CONTENTMGM,CHATBOT,ADVERTISING
0,0,0,0,1,0,0,0,0,0
1,1,0,1,1,0,0,0,0,0
2,2,1,0,1,0,0,1,0,0
3,3,0,0,1,0,0,0,0,0
4,4,1,1,1,0,0,1,0,0


As we can see we have a dataframe with 9 columns and 8 of this seems to be binary, 1 if has this category, 0 if not, but for the nexts analisys we do not need ID_customer.

In [8]:
print("WEBHOSTING:",sum(dataset.WEBHOSTING))
print("OFFICESUITE:",sum(dataset.OFFICESUITE))
print("SECURITY:",sum(dataset.SECURITY))
print("CLOUD_IAAS:",sum(dataset.CLOUD_IAAS))
print("CLOUD_PAAS:",sum(dataset.CLOUD_PAAS))
print("CONTENTMGM:",sum(dataset.CONTENTMGM))
print("CHATBOT:",sum(dataset.CHATBOT))
print("ADVERTISING:",sum(dataset.ADVERTISING))

WEBHOSTING: 274
OFFICESUITE: 176
SECURITY: 608
CLOUD_IAAS: 67
CLOUD_PAAS: 6
CONTENTMGM: 152
CHATBOT: 0
ADVERTISING: 9


In [9]:
#We remove the ID_customer column because is not necessary for the analysis
dataset1 = dataset.drop(columns = ["ID_customer"],axis = 1) 
print(dataset1.shape)

(1000, 8)


In [10]:
#We sum the value of all the rows
val = dataset1.WEBHOSTING.values + dataset1.OFFICESUITE.values + dataset1.SECURITY.values + dataset1.CLOUD_IAAS.values + dataset1.CLOUD_PAAS.values + dataset1.CONTENTMGM.values + dataset1.CHATBOT.values + dataset1.ADVERTISING.values
#We create a auxiliar function
def which_zero(lis):
    """
    Auxiliar function that return the index of the list where the value is 0
    """
    #List with the indexs
    indexs = []
    #We loop through the values
    for i,value in enumerate(lis):
        if value == 0:
            indexs.append(i)
    return indexs
#We drop the records with all 0
dataset2 = dataset1.drop(which_zero(val))
print(dataset2.shape)

(753, 8)


In [11]:
#Here we crate the transactions list
transactions = []
#We loop trhough the records
for recordnum, record in dataset2.iterrows():
    transaction = []
    #We loop trhough the colums
    for column in dataset2.columns:
        #If that value contains a one we add to the transaction the column name
        if record[column] == 1:
            transaction.append(column)
    transactions.append(transaction)
print(transactions)

[['SECURITY'], ['OFFICESUITE', 'SECURITY'], ['WEBHOSTING', 'SECURITY', 'CONTENTMGM'], ['SECURITY'], ['WEBHOSTING', 'OFFICESUITE', 'SECURITY', 'CONTENTMGM'], ['SECURITY', 'CLOUD_IAAS'], ['SECURITY'], ['SECURITY', 'CONTENTMGM'], ['SECURITY'], ['WEBHOSTING', 'SECURITY'], ['OFFICESUITE'], ['WEBHOSTING', 'SECURITY', 'CONTENTMGM'], ['WEBHOSTING', 'SECURITY'], ['SECURITY', 'ADVERTISING'], ['WEBHOSTING', 'OFFICESUITE', 'SECURITY'], ['WEBHOSTING', 'SECURITY'], ['SECURITY', 'CONTENTMGM'], ['SECURITY'], ['OFFICESUITE'], ['OFFICESUITE', 'SECURITY', 'CONTENTMGM'], ['SECURITY'], ['SECURITY', 'CONTENTMGM'], ['WEBHOSTING', 'OFFICESUITE', 'SECURITY'], ['WEBHOSTING'], ['SECURITY'], ['SECURITY'], ['OFFICESUITE', 'SECURITY', 'CLOUD_IAAS'], ['SECURITY'], ['SECURITY'], ['SECURITY'], ['WEBHOSTING'], ['SECURITY', 'CONTENTMGM'], ['OFFICESUITE', 'SECURITY', 'CLOUD_PAAS'], ['SECURITY'], ['SECURITY'], ['WEBHOSTING', 'SECURITY'], ['SECURITY'], ['SECURITY', 'CONTENTMGM'], ['OFFICESUITE'], ['WEBHOSTING'], ['SECURITY

As we can see now the transactions datatype is what we want for the apriory algorithm

# 3. Run the Apriori algorithm

In [12]:
#We calculate the apriori results with grater than 1 lift
results = list(apriori(transactions, min_support=0.03, min_confidence=0.8, min_lift=1))
print_apyori_output(results)

Rules involving itemset ['CONTENTMGM', 'SECURITY']
['CONTENTMGM'] => ['SECURITY'] (support=0.18, confidence=0.91, lift=1.12)

Rules involving itemset ['CONTENTMGM', 'OFFICESUITE', 'SECURITY']
['CONTENTMGM', 'OFFICESUITE'] => ['SECURITY'] (support=0.05, confidence=0.82, lift=1.01)

Rules involving itemset ['CONTENTMGM', 'WEBHOSTING', 'SECURITY']
['CONTENTMGM', 'WEBHOSTING'] => ['SECURITY'] (support=0.07, confidence=0.93, lift=1.15)



As we can see we have obtain 3 rules:

$ CONTENTMGM \Rightarrow SECURITY $

This first rule is saying that when we have CONTENTMGM we have SECURITY too with a confidence of 91%

$ [OFFICESUITE, CONTENTMGM] \Rightarrow SECURITY $

This second rule is saying that when we have [OFFICESUITE, CONTENTMGM] we have SECURITY too with a confidence of 82%

$ [WEBHOSTING, CONTENTMGM] \Rightarrow SECURITY $

This third rule is saying that when we have [WEBHOSTING, CONTENTMGM] we have SECURITY too with a confidence of 93%

**And all of them with a lift > 1**

In [14]:
#We look for the greater lift putting as a restriction and with lower values of min_sup and min_confidence
results = list(apriori(transactions, min_support=0.005, min_confidence=0.5, min_lift=3))
print_apyori_output(results)

Rules involving itemset ['CLOUD_PAAS', 'OFFICESUITE']
['CLOUD_PAAS'] => ['OFFICESUITE'] (support=0.01, confidence=0.83, lift=3.57)

Rules involving itemset ['CLOUD_IAAS', 'OFFICESUITE', 'CONTENTMGM']
['CLOUD_IAAS', 'CONTENTMGM'] => ['OFFICESUITE'] (support=0.01, confidence=0.82, lift=3.50)

Rules involving itemset ['CLOUD_IAAS', 'SECURITY', 'OFFICESUITE', 'CONTENTMGM']
['CLOUD_IAAS', 'CONTENTMGM'] => ['OFFICESUITE', 'SECURITY'] (support=0.01, confidence=0.55, lift=3.45)
['CLOUD_IAAS', 'SECURITY', 'CONTENTMGM'] => ['OFFICESUITE'] (support=0.01, confidence=0.75, lift=3.21)



As we can see we have obtain 4 rules:

$ CLOUD_PAAS \Rightarrow OFFICESUITE $

This first rule is saying that when we have CLOUD_PAAS we have OFFICESUITE too with a confidence of 83%

$ [CLOUD_IAAS, CONTENTMGM] \Rightarrow OFFICESUITE $

This second rule is saying that when we have [CLOUD_IAAS, CONTENTMGM] we have OFFICESUITE too with a confidence of 82%

$ [CLOUD_IAAS, CONTENTMGM] \Rightarrow [SECURITY,OFFICESUITE] $

This third rule is saying that when we have [CLOUD_IAAS, CONTENTMGM] we have [SECURITY,OFFICESUITE] too with a confidence of 55%

$ [CLOUD_IAAS, CONTENTMGM,SECURITY] \Rightarrow OFFICESUITE $

This third rule is saying that when we have [CLOUD_IAAS, CONTENTMGM,SECURITY] we have OFFICESUITE too with a confidence of 75%

**And all of them with a lift > 3**

In [16]:
#Aditional rules to understand the customers
results = list(apriori(transactions, min_support=0.005, min_confidence=0.4, min_lift=1))
print_apyori_output(results)

Rules involving itemset ['CLOUD_IAAS', 'OFFICESUITE']
['CLOUD_IAAS'] => ['OFFICESUITE'] (support=0.04, confidence=0.45, lift=1.92)

Rules involving itemset ['CLOUD_PAAS', 'OFFICESUITE']
['CLOUD_PAAS'] => ['OFFICESUITE'] (support=0.01, confidence=0.83, lift=3.57)

Rules involving itemset ['CONTENTMGM', 'SECURITY']
['CONTENTMGM'] => ['SECURITY'] (support=0.18, confidence=0.91, lift=1.12)

Rules involving itemset ['CLOUD_IAAS', 'OFFICESUITE', 'CONTENTMGM']
['CLOUD_IAAS', 'CONTENTMGM'] => ['OFFICESUITE'] (support=0.01, confidence=0.82, lift=3.50)

Rules involving itemset ['CLOUD_IAAS', 'OFFICESUITE', 'SECURITY']
['CLOUD_IAAS', 'SECURITY'] => ['OFFICESUITE'] (support=0.03, confidence=0.48, lift=2.05)

Rules involving itemset ['CLOUD_IAAS', 'OFFICESUITE', 'WEBHOSTING']
['CLOUD_IAAS', 'WEBHOSTING'] => ['OFFICESUITE'] (support=0.01, confidence=0.44, lift=1.90)

Rules involving itemset ['CLOUD_IAAS', 'WEBHOSTING', 'SECURITY']
['CLOUD_IAAS', 'WEBHOSTING'] => ['SECURITY'] (support=0.02, confidenc

As we can see in all this rules obtained, the customers that but CONTENTMGM buy SECURITY and this is related too with the OFFICESUITE.
And with WEBHOSTING we see that is related with the purchased mentioned before.

Knowing this rules now we can try to sugest some products to the customers:

For sure we have to take in count to recomend SECURITY if they buy CONTENTMGM and maybe recomend a offer of CONTENTMGM, OFFICESUITE, WEBHOSTING, SECURITY all together, as we can see in the rule:

Rules involving itemset ['CONTENTMGM', 'OFFICESUITE', 'WEBHOSTING', 'SECURITY']
['CONTENTMGM', 'OFFICESUITE', 'WEBHOSTING'] => ['SECURITY'] (support=0.02, confidence=0.87, lift=1.07)

<font size="+2" color="blue">Additional results: experiments on the bakery dataset</font>

In [17]:
#We load the raw data from github
url = "https://raw.githubusercontent.com/viktree/curly-octo-chainsaw/master/BreadBasket_DMS.csv"
raw_df = pd.read_csv(url)
raw_df.head(10)

,Date,Time,Transaction,Item
0,2016-10-30,09:58:11,1,Bread
1,2016-10-30,10:05:34,2,Scandinavian
2,2016-10-30,10:05:34,2,Scandinavian
3,2016-10-30,10:07:57,3,Hot chocolate
4,2016-10-30,10:07:57,3,Jam
5,2016-10-30,10:07:57,3,Cookies
6,2016-10-30,10:08:41,4,Muffin
7,2016-10-30,10:13:03,5,Coffee
8,2016-10-30,10:13:03,5,Pastry
9,2016-10-30,10:13:03,5,Bread


In [18]:
#We clean the data as in the notebook provide it
def cleanup_dataset(df):
    # Returns new dataset without NONE values in specified c
    df_none_entries = df.loc[df['Item']=='NONE',:]
    return df.drop(df_none_entries.index)

dataset = cleanup_dataset(raw_df)
dataset.head()

,Date,Time,Transaction,Item
0,2016-10-30,09:58:11,1,Bread
1,2016-10-30,10:05:34,2,Scandinavian
2,2016-10-30,10:05:34,2,Scandinavian
3,2016-10-30,10:07:57,3,Hot chocolate
4,2016-10-30,10:07:57,3,Jam


In [19]:
#We transform the data to a transaction type
transactions = []
for n_transact in dataset.Transaction.unique():
    transactions.append(dataset.Item.values[dataset["Transaction"] == n_transact])

In [20]:
#We calculate the apriori results
results = list(apriori(transactions, min_support=0.01, min_confidence=0.56, min_lift=1))
print_apyori_output(results)

Rules involving itemset ['Coffee', 'Medialuna']
['Medialuna'] => ['Coffee'] (support=0.04, confidence=0.57, lift=1.19)

Rules involving itemset ['Coffee', 'Spanish Brunch']
['Spanish Brunch'] => ['Coffee'] (support=0.01, confidence=0.60, lift=1.25)

Rules involving itemset ['Coffee', 'Toast']
['Toast'] => ['Coffee'] (support=0.02, confidence=0.70, lift=1.47)



With this 3 rules obtained, now we can discuss some posible decisions for the business:

For example we can create a offer of toast+coffee together with some discount, because we know with 70% of confidence that the people who buys a toast they buy a coffee too.
But we have the rule that if they buy a medialuna they but a coffee too, and the same with Spanish Brunch, so it seems that it is common to come here and take some food with coffee, so the offer could be for all food, or when they will buy ask for coffee with discount or without.

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>